<table style="font-size: 1em; padding: 0; margin: 0;">
<p style="border: 1px solid #ff5733; border-left: 15px solid #ff5733; padding: 10px; text-align:justify;">
    <strong style="color: #ff5733">Deliverable</strong>  
    <br/>The deliverable for Part 2 is a jupyter notebook showing a workflow to create test and training datasets, consisting of folders of imagery and corresponding label imagery, ready for training a semantic segmentation model in keras. This will mostly test your understanding the generic workflow for preparing a dataset to train and test a deep learning model, which is an essential component of the remaining Parts.
    </p>

In [1]:
import os
import urllib.request
import glob
import shutil
import rasterio

ModuleNotFoundError: No module named 'rasterio'

## Dataset 1

Text from the Buscomb Live Project:

Use the Sentinel-2 cloudless image service ("option 1") as a means to easily acquire cloudless mosaiced imagery from some of Earth's critical lakes that collectively provide drinking water to countless millions of people. You can use the website to explore and compare changes visually between 2016 and 2018 everywhere, and also 2017 for Europe only. This is the data you will be attempting to test your model (trained on another dataset that is introduced later).

Tip. If you use the download tool to obtain an image tile, you will see some structure in the URL. For example,
https://tiles.maps.eox.at/wms?service=wms&request=getmap&version=1.1.1&layers=s2cloudless-2018&bbox=10.232263646327286,-0.670557747166987,10.334917149745255,-0.5679042437490183&width=4096&height=4096&srs=epsg:4326

Let's break this down a little:

https://tiles.maps.eox.at/wms?service=wms&request=getmap&version=1.1.1: The root of the service that always stays the same. WMS stands for Web Map Service.
layers=s2cloudless2018: The layer to get. Other options are "s2cloudless" for 2016 imagery, and "s2cloudless2017" for 2017 imagery (Europe only).
bbox=10.232263646327286,-0.670557747166987,10.334917149745255,-0.5679042437490183: the bounding box coordinates (region of interest) in the form south Latitude, west Longitude, north Latitude, east Longitude.
width=4096&height=4096: the width and height of the image in number of pixels
srs=epsg:4326: the epsg code (coordinate projection system) that pertains to the bounding box coordinates. In this example the epsg code is 4326, which is GPS coordinates (WGS 84).

Download time-series of images from 17 critically endangered lake systems:
1. <a href="https://en.wikipedia.org/wiki/Lake_Poop%C3%B3">Lake Poopo, Bolivia</a>

2. <a href="https://en.wikipedia.org/wiki/Lake_Urmia">Lake Urmia, Iran</a> 
 
3. <a href="https://en.wikipedia.org/wiki/Lake_Mead">Lake Mead</a>/<a href="https://en.wikipedia.org/wiki/Lake_Mohave">Lake Mojave</a>, USA

4. <a href="https://en.wikipedia.org/wiki/Aral_Sea">Aral sea, Kazahkstan</a>

5. <a href="https://en.wikipedia.org/wiki/Lake_Copais">Lake Copais, Greece</a>

6. <a href="https://en.wikipedia.org/wiki/Ramganga_Dam">Lake Ramganga, India</a>
 
7. <a href="https://en.wikipedia.org/wiki/Qinghai_Lake">Qinghai Lake, China</a> 

8. <a href="https://en.wikipedia.org/wiki/Salton_Sea">Salton Sea, USA</a> 

9. <a href="https://earthobservatory.nasa.gov/images/8991/drying-of-lake-faguibine-mali">Lake Faguibine, Mali</a> 
 
10. <a href="https://en.wikipedia.org/wiki/Mono_Lake">Mono Lake, USA</a> 

11. <a href="https://en.wikipedia.org/wiki/Walker_Lake_(Nevada)">Walker Lake, USA</a>

12. <a href="https://en.wikipedia.org/wiki/Lake_Balaton">Lake Balaton, Hungary</a>

13. <a href="https://en.wikipedia.org/wiki/Lake_Koroneia">Lake Koroneia, Greece</a>

14. <a href="https://en.wikipedia.org/wiki/Lake_Salda">Lake Salda, Turkey</a>

15. <a href="https://en.wikipedia.org/wiki/Lake_Burdur">Lake Burdur, Turkey</a>

16. <a href="https://en.wikipedia.org/wiki/Lake_Mendocino">Lake Mendocino, USA</a>

17. <a href="https://en.wikipedia.org/wiki/Elephant_Butte_Reservoir">Elephant Butte Reservoir, USA</a>

You can read about some of these sites and their problems in a <a href="https://www.nationalgeographic.com/magazine/2018/03/drying-lakes-climate-change-global-warming-drought/">2018 National Geographic article</a>

#### Step 1: Download individual sites from S2 cloudless

In [2]:
def download_all_layers(root,layers,bbox,width,height,srs,site):
    #cycle through each layer to download
    for layer in layers:
        # construct a URL string 
        url = root+layer+bbox+width+height+srs
        #download into a filename constructed as "site_layer.jpg"
        urllib.request.urlretrieve(url, site+'_'+layer.split('=')[-1].split('&')[0]+'.jpg')

In [3]:
#Lake Poopo, Bolivia
root = "https://tiles.maps.eox.at/wms?service=wms&request=getmap&version=1.1.1&"
site='poopo'
bbox = 'bbox=-68.66848367000537,-19.687928531849003,-66.67924128546656,-17.8774477409051&'
width = 'width=4096&'
height = 'height=4096&'
layers = ['layers=s2cloudless&', 'layers=s2cloudless-2018&']
srs = 'srs=epsg:4326'

In [4]:
download_all_layers(root,layers,bbox,width,height,srs,site)

In [5]:
#Lake Urmia, Iran
bbox = 'bbox=44.587725529095295,36.86436828406643,46.230181583782795,38.50682433875393&'
layers = ['layers=s2cloudless&', 'layers=s2cloudless-2017&', 'layers=s2cloudless-2018&']
download_all_layers(root,layers,bbox,width,height,srs,'urmia')

#Lake Mead/ Lake Mojave, USA
bbox = 'bbox=-115.42507235769445,34.94273489926993,-113.78261630300695,36.58519095395743&'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
download_all_layers(root,layers,bbox,width,height,srs,'mead_mojave')

#Aral Sea, Kazahkstan
bbox = 'bbox=58.032853536637845,43.955292007325,61.317765646012845,47.2402041167&'
download_all_layers(root,layers,bbox,width,height,srs,'aral_sea')

#Copais Lake, Greece
bbox = 'bbox=23.214268551013436,38.36067354565393,23.350911007068124,38.4633270490719&'
layers = ['layers=s2cloudless&', 'layers=s2cloudless-2017&', 'layers=s2cloudless-2018&']
download_all_layers(root,layers,bbox,width,height,srs,'copais')

#Ramganga Lake, India
bbox = 'bbox=78.63716910452058,29.468167756293038,78.91045401662996,29.673474763128976&'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
download_all_layers(root,layers,bbox,width,height,srs,'ramganga')

#Qinghai Lake, China 
bbox = 'bbox=99.56471967977474,35.70094613663666,101.20168257039974,37.34340219132416&'
download_all_layers(root,layers,bbox,width,height,srs,'qinghai')

#Salton Sea, USA 
bbox = 'bbox=-116.21928854749297,32.903320741965295,-115.40080710218047,33.724548769309045&'
download_all_layers(root,layers,bbox,width,height,srs,'salton_sea')

# Lake Faguibine, Mali
bbox = 'bbox=-4.656089323623854,14.959306189862815,-3.0191264329988536,16.601762244550315&'
width = 'width=4082&'
download_all_layers(root,layers,bbox,width,height,srs,'faguibine')

# Mono Lake, USA
bbox = 'bbox=-119.22357779576633,37.797192162208084,-118.81433707311008,38.20780617587996&'
download_all_layers(root,layers,bbox,width,height,srs,'mono')

# Walker Lake, USA
bbox = 'bbox=-118.82841344222616,38.58649115903216,-118.62379308089804,38.791798165868094&'
download_all_layers(root,layers,bbox,width,height,srs,'walker')

# Lake Balaton, Hungary
width = 'width=4096&'
height = 'height=3084&'
layers = ['layers=s2cloudless&', 'layers=s2cloudless-2017&', 'layers=s2cloudless-2018&']
bbox = 'bbox=17.138306471163226,46.438781725159295,18.231446119600726,47.260009752503045&'
download_all_layers(root,layers,bbox,width,height,srs,'balaton')

# Lake Koroneia, Greece
bbox = 'bbox=23.34167375183668,40.56321556296156,23.614958663946055,40.7685225697975&'
download_all_layers(root,layers,bbox,width,height,srs,'koroneia')

# Lake Salda, Turkey
width = 'width=4082&'
height = 'height=4096&'
bbox = 'bbox=29.6337346568769,37.49763563391751,29.736044837540963,37.60028913733548&'
download_all_layers(root,layers,bbox,width,height,srs,'salda')

# Lake Burdur, Turkey
width = 'width=4096&'
height = 'height=2312&'
bbox = 'bbox=30.008970333642345,37.6345355820376,30.373579098290783,37.83984258887354&'
download_all_layers(root,layers,bbox,width,height,srs,'burdur')

# Lake Mendocino, USA
width = 'width=4082&'
height = 'height=4096&'
bbox = 'bbox=-123.19613698824355,39.189371563341254,-123.14498189791152,39.24069831505024&'
layers = ['layers=s2cloudless&','layers=s2cloudless-2018&']
download_all_layers(root,layers,bbox,width,height,srs,'mendocino')

# Elephant Butte Reservoir, USA
bbox = 'bbox=-107.23803910498334,33.14614806405013,-107.13572892431928,33.2488015674681&'
download_all_layers(root,layers,bbox,width,height,srs,'elephant_butte')

In [8]:
data_path = 'data/s2cloudless_imagery'

try:
    os.mkdir(data_path)
except:
    pass

try:
    for f in glob.glob('*.jpg'):
        #move to the new directory
        if f.endswith('s2cloudless.jpg'): #2016 imagery
            shutil.move(f, os.path.join(data_path,f.replace('s2cloudless.jpg','s2cloudless-2016.jpg')))
        else: #2017 or 2018 imagery
            shutil.move(f, data_path)
except:
    pass        

#### Step 2: View downloaded images

In [ ]:
# define a list of sites
sites = ['aral_sea','balaton','burdur','copais','elephant_butte',
         'faguibine','koroneia','mead_mojave','mendocino','mono',
         'poopo','qinghai','ramganga','salda','urmia','walker']